<a href="https://colab.research.google.com/github/lulas98/ETL/blob/main/examen2021v2LV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Cargamos distintas librerias

In [5]:
from pyspark.sql.functions import col
import pandas as pd
from pyspark.sql.functions import mean as mean_
import pyspark.sql.functions as F




Cargamos los Datos

In [7]:
 RawData = spark.read.option("delimiter" , ";").option("header", "True").csv("BDpracticafinalCSV.csv")
 RawData.show()

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|      

Renombramos las columnas


In [8]:
data = RawData.select(col("Order").alias("Order"),col("MS Zoning").alias("MS_Zoning"), col("Lot Frontage").alias("Lot_Frontage"),col("Lot Area").alias("Lot_Area"), col("Land Contour").alias("Land_Contour"),col("Land Slope").alias("Land_Slope"), col("Overall Qual").alias("Overall_Qual"),col("Year Built").alias("Year_Built"), col("Year Remod/Add").alias("Year_Remod_Add"),col("Mas Vnr Type").alias("Mas_Vnr_Type"), col("Exter Qual").alias("Exter_Qual"),col("Bsmt Cond").alias("Bsmt_Cond"), col("Total Bsmt SF").alias("Total_Bsmt_SF"),col("Heating QC").alias("Heating_QC"), col("Central Air").alias("Central_Air"),col("Full Bath").alias("Full_Bath"), col("Half Bath").alias("Half_Bath"),col("Bedroom AbvGr").alias("Bedroom_AbvGr"), col("Kitchen AbvGr").alias("Kitchen_AbvGr"),col("Kitchen Qual").alias("Kitchen_Qual"),col("TotRms AbvGrd").alias("TotRms_AbvGrd"), col("Garage Cars").alias("Garage_Cars"),col("Garage Area").alias("Garage_Area"), col("Garage Cond").alias("Garage_Cond"),col("Pool Area").alias("Pool_Area"), col("Pool QC").alias("Pool_QC"),col("Mo Sold").alias("Mo_Sold"), col("Yr Sold").alias("Yr_Sold"),col("SalePrice").alias("SalePrice"))
data.show()

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS_Zoning|Lot_Frontage|Lot_Area|Land_Contour|Land_Slope|Overall_Qual|Year_Built|Year_Remod_Add|Mas_Vnr_Type|Exter_Qual|Bsmt_Cond|Total_Bsmt_SF|Heating_QC|Central_Air|Full_Bath|Half_Bath|Bedroom_AbvGr|Kitchen_AbvGr|Kitchen_Qual|TotRms_AbvGrd|Garage_Cars|Garage_Area|Garage_Cond|Pool_Area|Pool_QC|Mo_Sold|Yr_Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|      

Modificamos los datos a el formato acorde a su tipo de dato

In [9]:
data = data.select(data.Order.cast("int"),
data.Lot_Area.cast("int"),
data.MS_Zoning.cast("string"),
data.Lot_Frontage.cast("int"),
data.Land_Contour.cast("string"),
data.Land_Slope.cast("string"),
data.Overall_Qual.cast("string"),
data.Year_Built.cast("int"),
data.Year_Remod_Add.cast("int"),
data.Mas_Vnr_Type.cast("string"),
data.Exter_Qual.cast("string"),
data.Bsmt_Cond.cast("string"),
data.Total_Bsmt_SF.cast("int"),
data.Heating_QC.cast("string"),
data.Central_Air.cast("string"),
data.Full_Bath.cast("int"),
data.Half_Bath.cast("int"),
data.Bedroom_AbvGr.cast("int"),
data.Kitchen_AbvGr.cast("int"),
data.Kitchen_Qual.cast("string"),
data.TotRms_AbvGrd.cast("int"),
data.Garage_Cars.cast("int"),
data.Garage_Area.cast("int"),
data.Garage_Cond.cast("string"),
data.Pool_Area.cast("int"),
data.Pool_QC.cast("string"),
data.Mo_Sold.cast("int"),
data.Yr_Sold.cast("int"),
data.SalePrice.cast("int"))


In [10]:
data.show()

+-----+--------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|Lot_Area|MS_Zoning|Lot_Frontage|Land_Contour|Land_Slope|Overall_Qual|Year_Built|Year_Remod_Add|Mas_Vnr_Type|Exter_Qual|Bsmt_Cond|Total_Bsmt_SF|Heating_QC|Central_Air|Full_Bath|Half_Bath|Bedroom_AbvGr|Kitchen_AbvGr|Kitchen_Qual|TotRms_AbvGrd|Garage_Cars|Garage_Area|Garage_Cond|Pool_Area|Pool_QC|Mo_Sold|Yr_Sold|SalePrice|
+-----+--------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|   317

Primero parseamos los datos, pues existen varios datos que significan los mismo pero están escritos de forma distinta, como es el caso de (RL, rL, Rl)

In [11]:
data = data.withColumn("MS_Zoning", F.when(F.col("Ms_Zoning") == "Rl", "RL").when(F.col("Ms_Zoning") == "rl", "RL").when(F.col("Ms_Zoning") == "rL" , "RL").otherwise(F.col("Ms_Zoning")))

Filtramos para quedarnos solo con las zonas residenciales de baja, media, y alta densidad

In [12]:
data = data.filter(" Ms_Zoning == 'RL'  or Ms_Zoning == 'RH' or Ms_Zoning == 'RM'")

data.show()

+-----+--------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|Lot_Area|MS_Zoning|Lot_Frontage|Land_Contour|Land_Slope|Overall_Qual|Year_Built|Year_Remod_Add|Mas_Vnr_Type|Exter_Qual|Bsmt_Cond|Total_Bsmt_SF|Heating_QC|Central_Air|Full_Bath|Half_Bath|Bedroom_AbvGr|Kitchen_AbvGr|Kitchen_Qual|TotRms_AbvGrd|Garage_Cars|Garage_Area|Garage_Cond|Pool_Area|Pool_QC|Mo_Sold|Yr_Sold|SalePrice|
+-----+--------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|   317

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

In [13]:
data.count()

2768

In [14]:
DatosDuplicados = data.dropDuplicates()
DatosDuplicados.count()


2762

In [15]:
data = data.dropDuplicates()

 Existen 6 datos duplicados, lo cual es muy poco teniendo en cuenta el tamaño del dataset. Aún así no tiene sentido que se repitan, ya que debería haber solo una orden por cada casa, es por ello que podemos eliminarlas.

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

# Total de Inmuebles

In [16]:
data.groupBy("MS_Zoning").count().show()

+---------+-----+
|MS_Zoning|count|
+---------+-----+
|       RH|   27|
|       RL| 2273|
|       RM|  462|
+---------+-----+



 # Precio Medio de cada Zona

In [17]:
data.groupBy("MS_Zoning").agg(mean_("SalePrice")).show()

+---------+------------------+
|MS_Zoning|    avg(SalePrice)|
+---------+------------------+
|       RH|136419.77777777778|
|       RL|191283.25164980203|
|       RM|126781.39393939394|
+---------+------------------+



## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

#Total Bsmt SF

In [18]:
decades = data.withColumn("Year_Built", F.when(F.col("Year_Built") <= 1880 , "1870-1880").when(F.col("Year_Built").between (1880, 1890) , "1880-1890").when(F.col("Year_Built").between (1890, 1900) , "1890-1900").when(F.col("Year_Built").between (1900, 1910) , "1900-1910").when(F.col("Year_Built").between (1910, 1920) , "1910-1920").when(F.col("Year_Built").between (1920, 1930) , "1920-1930").when(F.col("Year_Built").between (1930, 1940) , "1930-1940").when(F.col("Year_Built").between (1940, 1950) , "1940-1950").when(F.col("Year_Built").between (1950, 1960) , "1950-1960").when(F.col("Year_Built").between (1960, 1970) , "1960-1970").when(F.col("Year_Built").between (1970, 1980) , "1970-1980").when(F.col("Year_Built").between (1980, 1990) , "1980-1990").when(F.col("Year_Built").between (1990, 2000) , "1990-2000").when(F.col("Year_Built").between (2000, 2010) , "2000-2010").when(F.col("Year_Built").between (2010, 2020) , "2010-2020").otherwise(F.col("Year_Built")))


In [19]:
sotanos = decades.groupBy("Year_Built").agg(mean_("Total_Bsmt_SF"))
sotanos.show(20)

+----------+------------------+
|Year_Built|avg(Total_Bsmt_SF)|
+----------+------------------+
| 1870-1880|           829.625|
| 1940-1950| 709.5894039735099|
| 1890-1900| 763.1612903225806|
| 1900-1910| 726.4285714285714|
| 1930-1940| 779.9915254237288|
| 1980-1990|1156.4642857142858|
| 1920-1930| 832.8711656441718|
| 1970-1980| 958.1260744985674|
| 1950-1960|1010.1167664670659|
| 2000-2010|1361.6677265500796|
| 1960-1970| 1052.756906077348|
| 1910-1920| 807.3275862068965|
| 1880-1890|             803.8|
| 1990-2000|1130.4363636363637|
+----------+------------------+



# Heating QC

Le vamos a dar puntos a cada tipo de valoración de la calidad de la calefacción, para poder hacer una media por décadas. El máximo de puntuación es 5, si es Excelente, y la menor es 1, que ocurre cuando es pobre.

In [21]:
puntos = decades.withColumn("Heating_QC", F.when(F.col("Heating_QC") == "Ex" , 5).when(F.col("Heating_QC") == "Gd" , 4).when(F.col("Heating_QC") == "TA" , 3).when(F.col("Heating_QC") == "Fa", 2).when(F.col("Heating_QC") == "Po", 1).otherwise(0))


In [22]:
sumaPuntos = puntos.groupBy("Year_Built").agg(mean_("Heating_QC"))
sumaPuntos.orderBy("avg(Heating_QC)", ascending = [0]).show(5)

+----------+------------------+
|Year_Built|   avg(Heating_QC)|
+----------+------------------+
| 2000-2010| 4.960254372019078|
| 1990-2000| 4.775757575757575|
| 1880-1890|               4.0|
| 1890-1900| 3.967741935483871|
| 1930-1940|3.9661016949152543|
+----------+------------------+
only showing top 5 rows



Podemos comprobar que los años donde mejor puntuación en acondicionado para el frio son  las construidas entre la década del 2000-2010

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Antes de nada hay que transformar la variable Lot_Area de pies cuadrados a metros cuadrados, para calcular así el precio por metro cuadrado. Un pie equivale a 0.3048 metros

In [23]:
dataM2 = data.withColumn("Lot_Area", col("Lot_Area")*0.09290304)

In [24]:
dataM2 = dataM2.withColumn("Price_per_M2", col("SalePrice")/F.col("Lot_Area"))

In [25]:
data_filter = dataM2.filter("Year_Built == '2009'").orderBy(["Price_per_M2"],ascending=[0])
data_filter.show(10)

+-----+------------------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+------------------+
|Order|          Lot_Area|MS_Zoning|Lot_Frontage|Land_Contour|Land_Slope|Overall_Qual|Year_Built|Year_Remod_Add|Mas_Vnr_Type|Exter_Qual|Bsmt_Cond|Total_Bsmt_SF|Heating_QC|Central_Air|Full_Bath|Half_Bath|Bedroom_AbvGr|Kitchen_AbvGr|Kitchen_Qual|TotRms_AbvGrd|Garage_Cars|Garage_Area|Garage_Cond|Pool_Area|Pool_QC|Mo_Sold|Yr_Sold|SalePrice|      Price_per_M2|
+-----+------------------+---------+------------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+--

## Media anual por zonas del precio de venta y metros cuadrados.

In [26]:
data_zone = dataM2.groupBy("Year_Built", "MS_Zoning").avg("SalePrice", "Lot_Area")
data_zone.orderBy(["Year_Built"], ascending = [0]).show()

+----------+---------+------------------+------------------+
|Year_Built|MS_Zoning|    avg(SalePrice)|     avg(Lot_Area)|
+----------+---------+------------------+------------------+
|      2010|       RL|          290716.0| 871.1518060800001|
|      2009|       RL|308183.73333333334|    1074.615657216|
|      2008|       RL|331959.25641025644|1273.9222164184616|
|      2008|       RM|          267750.0|      716.42179296|
|      2007|       RM|          262232.0|      357.02638272|
|      2007|       RL|          271814.7|    1007.685210432|
|      2006|       RM|          305000.0|      479.28678336|
|      2006|       RL| 265620.2831858407|1023.5201910371685|
|      2005|       RM|176571.42857142858| 368.9312437028571|
|      2005|       RL|251356.73333333334| 961.2956257920004|
|      2004|       RM|          179546.5|362.57269420800003|
|      2004|       RL|     231581.484375|1007.7889521600001|
|      2003|       RM|         146093.75|      540.95117616|
|      2003|       RL|24

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

In [27]:
data_materials = data.groupBy("Mas_Vnr_Type").sum("SalePrice")
data_materials.show()


+------------+--------------+
|Mas_Vnr_Type|sum(SalePrice)|
+------------+--------------+
|        None|     250538498|
|      BrkCmn|       3504975|
|        null|       3831658|
|       Stone|      60175477|
|     BrkFace|     178912561|
|      CBlock|         80000|
+------------+--------------+



In [28]:
data_materials = data.groupBy("Mas_Vnr_Type").avg("SalePrice")
data_materials.show()

+------------+------------------+
|Mas_Vnr_Type|    avg(SalePrice)|
+------------+------------------+
|        None| 152488.4345709069|
|      BrkCmn|          140199.0|
|        null|        239478.625|
|       Stone| 262775.0087336245|
|     BrkFace|210981.79363207548|
|      CBlock|           80000.0|
+------------+------------------+



Si miramos valores absolutos lo que mas recauda es el  ladrillo descubierto, con mas de 178.912.561 dolares recaudados, frente a los 60.175.477 de la piedra. El ladrillo común por su parte recaudó solamente 3.504.975. 

La diferencia entre la media del revestimiento del ladrillo normal (BrkCmn) (140.199) y la piedra (Stone) (262.775)  es de mas de 120.000 euros, pero si consideramos a el ladrillo visto (BrkFace) (210.981) la diferencia no es tan significativa, 50.000. 
El revestimiento de ladrillo en general recauda mucho más. Aún así el revestimiento de piedra recauda más de media. 

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

In [29]:
data2cocinas = data.filter("Kitchen_AbvGr = '2'").filter("Garage_Cars >= '2'") 
data1cocina = data.filter("Kitchen_AbvGr = '1'").filter("Garage_Cars = '1'") 


In [30]:

data2cocinas.select(mean_("SalePrice")).show()

+------------------+
|    avg(SalePrice)|
+------------------+
|145124.27906976745|
+------------------+



In [31]:
data1cocina.select(mean_("SalePrice")).show()

+------------------+
|    avg(SalePrice)|
+------------------+
|128121.99337748345|
+------------------+



De media las viviendas con 2 cocinas y dos o más plazas de garaje son casi 20.000 euros más caras.

In [37]:
data2cocinas.approxQuantile("SalePrice", [0.25,0.5,0.75], 0.1)


[118858.0, 135000.0, 171000.0]

In [38]:
data1cocina.approxQuantile("SalePrice", [0.25,0.5,0.75,], 0.1)

[111750.0, 131000.0, 150000.0]

Los puntos de corte de los cuartiles casi no tienen una diferencia significativa, siendo los cuartiles de las casas con 2 cocinas y 2 o más coches  superiores a los cuartiles de las casas con 1 cocina y 1 coche, con una diferencia en los cortes que van aumentando conforme aumenta conforme subimos de cuartil, teniendo el corte inferior para el cuarto cuartil una diferencia entre ambos de más de 21.000

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Calculamos los cuartiles del precio

In [508]:
data.approxQuantile("SalePrice", [0.25,0.5,0.75], 0.1)

[157000.0, 175900.0, 287602.0]

Separamos por cuartiles los datos y vemos su correlación con el número de plazas de garaje.

In [39]:
primer_cuartil = data.filter("SalePrice <= '157000'")
primer_cuartil.corr("SalePrice", "Garage_Cars")


0.363979308154081

In [40]:
segundo_cuartil = data.filter(F.col("SalePrice").between (157000, 175900))
segundo_cuartil.corr("SalePrice", "Garage_Cars")

0.28722470613427026

In [41]:
tercer_cuartil = data.filter(F.col("SalePrice").between (175900, 287602))
tercer_cuartil.corr("SalePrice", "Garage_Cars")


0.3272084886722844

In [42]:
cuarto_cuartil = data.filter("SalePrice >= '287602'")
cuarto_cuartil.corr("SalePrice", "Garage_Cars")

0.2381481587736009

Podemos comprobar como la correlación entre el número de plazas de garaje y el precio de la vivienda varía significamente dependiendo del cuartil. La correlación en el primer cuartil es del 36%, mientras que la del segundo cuartil desciende a 28%. Esto tiene una explicación lógica, y es que mientras estás en el primer cuartil, es muy fácil que el número de plazas de garaje sea solo 0 o 1, y claramente en precios bajos una plaza de garaje extra aporta numeroso valor a la vivienda, aumentando la correlación. En el segundo cuartil los datos son más caóticos, ya que el rango de precios en el que fluctuamos existen otras variables mucho más influyentes.
El tercer cuartil asciende al 32% de correlación, ya que con el nuevo rango de precio, uno superior a la mediana (entre 175.900-287.602), la existencia de plaza de garaje casi es una obligación, y no fluctua tanto, siendo solo o 1 o 2.
Por último el último cuartil tiene una correlación de más del 23%. Este enorme descenso se debe a que en esos volúmenes de dinero, suele ser indiferente 2,3 o 4 plazas de garaje. Son otras las variables que influyen más en el precio, como lo puede ser la existencia de una piscina. 



## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta